In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import datetime

# ====================================
# 1) Mixed Precision & Basic Setup
# ====================================
tf.keras.mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision enabled:", tf.keras.mixed_precision.global_policy())

# ====================================
# 2) CONFIGURATION & PATHS
# ====================================
image_dir = "/kaggle/input/trees-nineteen-dataset"
save_dir = "/kaggle/working"
os.makedirs(save_dir, exist_ok=True)

model_prefix = os.path.join(save_dir, "model_epoch_")
latest_model_path = os.path.join(save_dir, "latest_model.keras")
history_path = os.path.join(save_dir, "history.json")
class_indices_path = os.path.join(save_dir, "class_indices.json")
log_dir = os.path.join(save_dir, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

weights_file = "/kaggle/input/efficientnetb3/efficientnetb3_notop.h5"
if not os.path.exists(weights_file):
    raise FileNotFoundError("Weights file not found! Please upload EfficientNetB3 weights.")
else:
    print("Using local weights file:", weights_file)

batch_size = 32
input_shape = (224, 224, 3)
initial_epochs = 3
fine_tune_epochs = 5
base_learning_rate = 0.0005
fine_tune_learning_rate = 1e-6

# ====================================
# 3) DETECT CLASSES & BUILD CLASS INDICES
# ====================================
detected_classes = sorted(
    d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))
)

if os.path.exists(class_indices_path):
    with open(class_indices_path, "r") as f:
        class_indices = json.load(f)
else:
    class_indices = {name: i for i, name in enumerate(detected_classes)}
    with open(class_indices_path, "w") as f:
        json.dump(class_indices, f)
num_classes = len(class_indices)
print("Class indices:", class_indices)

# ====================================
# 4) OPTIMIZED DATA LOADING USING `tf.data`
# ====================================
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))  # Resize to model input size
    image = tf.keras.applications.efficientnet.preprocess_input(image)  # Normalize [-1, 1]
    return image, label

def load_data(subset):
    datagen = tf.keras.preprocessing.image_dataset_from_directory(
        image_dir,
        image_size=(224, 224),
        batch_size=batch_size,
        validation_split=0.2,
        subset=subset,
        seed=42,
        label_mode='categorical'
    )
    return datagen.map(preprocess_image).cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

train_dataset = load_data("training")
val_dataset = load_data("validation")

# ====================================
# 5) DEFINE MODEL CREATION FUNCTION
# ====================================
def create_model(fine_tune=False):
    base_model = keras.applications.EfficientNetB3(
        include_top=False,
        weights=weights_file,
        input_shape=input_shape
    )

    if fine_tune:
        base_model.trainable = True
        fine_tune_at = len(base_model.layers) - 50  # Unfreeze last 50 layers
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
        learning_rate = fine_tune_learning_rate
    else:
        base_model.trainable = False
        learning_rate = base_learning_rate

    model = keras.Sequential([
        keras.Input(shape=input_shape),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])

    optimizer = keras.optimizers.AdamW(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ====================================
# 6) LOAD OR CREATE MODEL
# ====================================
if os.path.exists(latest_model_path):
    model = keras.models.load_model(latest_model_path)
    print("✅ Loaded latest model from disk.")
else:
    model = create_model()
    print("🚀 Created new model from scratch.")

# ====================================
# 7) TRAINING FUNCTION WITH PER-EPOCH SAVING
# ====================================
def train_model(model, epochs, start_epoch=1):
    for epoch in range(start_epoch, start_epoch + epochs):
        print(f"\n🔄 Training epoch {epoch}/{start_epoch + epochs - 1}...")
        model.fit(train_dataset, validation_data=val_dataset, epochs=1)

        model_epoch_path = f"{model_prefix}{epoch}.keras"
        model.save(model_epoch_path)
        model.save(latest_model_path)
        print(f"✅ Model saved: {model_epoch_path}")

    print("✅ Training session complete!")

# ====================================
# 8) INITIAL TRAINING (SAVES EVERY EPOCH)
# ====================================
print("\n=== Initial Training Phase ===")
train_model(model, epochs=2)  # Run only 2 epochs per Kaggle session

# ====================================
# 9) HOW TO RESUME TRAINING NEXT TIME
# ====================================
# 1. Upload the latest saved model (e.g., model_epoch_2.keras) to Kaggle input.
# 2. Update 'latest_model_path' to point to the uploaded model.
# 3. Restart training from the next epoch.


Mixed precision enabled: <DTypePolicy "mixed_float16">
Using local weights file: /kaggle/input/efficientnetb3/efficientnetb3_notop.h5
Class indices: {'Azadirachta_indica': 0, 'Baccharis_salicifolia': 1, 'Betula_pendula': 2, 'Camphora_officinarum': 3, 'Ceiba_pentandra': 4, 'Corylus_avellana': 5, 'Delonix_regia': 6, 'Eucalyptus_globulus': 7, 'Ficus_carica': 8, 'Ficus_microcarpa': 9, 'Juglans_regia': 10, 'Leucaena_leucocephala': 11, 'Ligustrum_lucidum': 12, 'Ligustrum_ovalifolium': 13, 'Malus_domestica': 14, 'Mangifera_indica': 15, 'Murraya_paniculata': 16, 'Olea_europaea': 17, 'Paulownia_tomentosa': 18, 'Pinus_halepensis': 19, 'Psidium_guajava': 20, 'Spathodea_campanulata': 21, 'Tecoma_stans': 22, 'Tilia_cordata': 23, 'Ulmus_parvifolia': 24, 'Washingtonia_robusta': 25, 'unknown': 26}
Found 132025 files belonging to 27 classes.
Using 105620 files for training.
Found 132025 files belonging to 27 classes.
Using 26405 files for validation.
🚀 Created new model from scratch.

=== Initial Train

In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import mixed_precision
import datetime

# ====================================
# 1) Mixed Precision & Basic Setup
# ====================================
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision enabled:", mixed_precision.global_policy())

# ====================================
# 2) CONFIGURATION & PATHS
# ====================================
image_dir = "/kaggle/input/trees-nineteen-dataset"
save_dir = "/kaggle/working"
os.makedirs(save_dir, exist_ok=True)

model_prefix = os.path.join(save_dir, "model_epoch_")
latest_model_path = os.path.join(save_dir, "latest_model.keras")
history_path = os.path.join(save_dir, "history.json")
class_indices_path = os.path.join(save_dir, "class_indices.json")
log_dir = os.path.join(save_dir, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

weights_file = "/kaggle/input/efficientnetb3/efficientnetb3_notop.h5"
if not os.path.exists(weights_file):
    raise FileNotFoundError("Weights file not found! Please upload EfficientNetB3 weights.")
else:
    print("Using local weights file:", weights_file)

batch_size = 64
input_shape = (224, 224, 3)
initial_epochs = 3
fine_tune_epochs = 5
base_learning_rate = 0.0005
fine_tune_learning_rate = 1e-6

# ====================================
# 3) DETECT CLASSES & BUILD CLASS INDICES
# ====================================
detected_classes = sorted(
    d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))
)

if os.path.exists(class_indices_path):
    with open(class_indices_path, "r") as f:
        class_indices = json.load(f)
else:
    class_indices = {name: i for i, name in enumerate(detected_classes)}
    with open(class_indices_path, "w") as f:
        json.dump(class_indices, f)
num_classes = len(class_indices)
print("Class indices:", class_indices)

# ====================================
# 4) SET UP DATA GENERATORS
# ====================================
from tensorflow.keras.applications.efficientnet import preprocess_input

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Normalizes to [-1, 1]
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode="nearest",
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=list(class_indices.keys())
)

val_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=list(class_indices.keys())
)

# ====================================
# 5) DEFINE MODEL CREATION FUNCTION
# ====================================
def create_model(fine_tune=False):
    base_model = keras.applications.EfficientNetB3(
        include_top=False,
        weights=weights_file,
        input_shape=input_shape
    )

    if fine_tune:
        base_model.trainable = True
        fine_tune_at = len(base_model.layers) - 50  # Unfreeze last 50 layers
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
        learning_rate = fine_tune_learning_rate
    else:
        base_model.trainable = False
        learning_rate = base_learning_rate

    model = keras.Sequential([
        keras.Input(shape=input_shape),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])

    optimizer = keras.optimizers.AdamW(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ====================================
# 6) LOAD OR CREATE MODEL
# ====================================
if os.path.exists(latest_model_path):
    model = keras.models.load_model(latest_model_path)
    print("✅ Loaded latest model from disk.")
else:
    model = create_model()
    print("🚀 Created new model from scratch.")

# ====================================
# 7) TRAINING FUNCTION WITH PER-EPOCH SAVING
# ====================================
def train_model(model, epochs, start_epoch=1):
    for epoch in range(start_epoch, start_epoch + epochs):
        print(f"\n🔄 Training epoch {epoch}/{start_epoch + epochs - 1}...")
        model.fit(train_generator, validation_data=val_generator, epochs=1)
        
        model_epoch_path = f"{model_prefix}{epoch}.keras"
        model.save(model_epoch_path)
        model.save(latest_model_path)
        print(f"✅ Model saved: {model_epoch_path}")
        
    print("✅ Training session complete!")

# , start_epoch=3
# ====================================
# 8) INITIAL TRAINING (SAVES EVERY EPOCH)
# ====================================
print("\n=== Initial Training Phase ===")
train_model(model, epochs=2)  # Run only 2 epochs per Kaggle session

# ====================================
# 9) HOW TO RESUME TRAINING NEXT TIME
# ====================================
# 1. Upload the latest saved model (e.g., model_epoch_2.keras) to Kaggle input.
# 2. Update 'latest_model_path' to point to the uploaded model.
# 3. Restart training from the next epoch.


In [3]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# ====================================
# 1) Basic Setup & Paths
# ====================================
tf.keras.mixed_precision.set_global_policy('mixed_float16')
print("✅ Mixed precision enabled")

image_dir = "/kaggle/input/trees-nineteen-dataset"
weights_path = "/kaggle/input/efficientnetb3/efficientnetb3_notop.h5"  # Uploaded weights
save_dir = "/kaggle/working"
os.makedirs(save_dir, exist_ok=True)

latest_model_path = os.path.join(save_dir, "latest_model.keras")
class_indices_path = os.path.join(save_dir, "class_indices.json")

batch_size = 16  # ✅ Adjusted for better memory usage
input_shape = (224, 224, 3)
epochs = 3
learning_rate = 0.0005
fine_tune_layers = 50  # ✅ Unfreeze last 50 layers

# ====================================
# 2) Detect Classes
# ====================================
detected_classes = sorted(
    d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))
)

if os.path.exists(class_indices_path):
    with open(class_indices_path, "r") as f:
        class_indices = json.load(f)
else:
    class_indices = {name: i for i, name in enumerate(detected_classes)}
    with open(class_indices_path, "w") as f:
        json.dump(class_indices, f)
num_classes = len(class_indices)
print(f"📌 Detected {num_classes} classes")

# ====================================
# 3) Optimized Data Pipeline
# ====================================
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.efficientnet.preprocess_input(image)
    return image, label

def load_data(subset):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        image_dir,
        image_size=(224, 224),
        batch_size=batch_size,
        validation_split=0.2,
        subset=subset,
        seed=42,
        label_mode='categorical'
    )
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    
    if subset == "training":
        dataset = dataset.prefetch(tf.data.AUTOTUNE)  # ✅ No `cache()` to save RAM
    else:
        dataset = dataset.cache().prefetch(tf.data.AUTOTUNE)  # ✅ Cache only validation

    return dataset

train_dataset = load_data("training")
val_dataset = load_data("validation")

# ====================================
# 4) Model Creation (Load Weights Manually)
# ====================================
def create_model():
    base_model = keras.applications.EfficientNetB3(
        include_top=False,
        weights=None,  # ✅ Avoids Kaggle download restrictions
        input_shape=input_shape
    )

    # ✅ Load weights from uploaded file
    if os.path.exists(weights_path):
        base_model.load_weights(weights_path)
        print("✅ Loaded EfficientNetB3 pretrained weights")
    else:
        print("⚠️ Warning: Pretrained weights not found! Using random initialization.")

    base_model.trainable = True
    for layer in base_model.layers[:-fine_tune_layers]:  
        layer.trainable = False  # ✅ Freeze early layers

    model = keras.Sequential([
        keras.Input(shape=input_shape),
        base_model,
        layers.MaxPooling2D(pool_size=(2, 2)),  # ✅ Saves memory
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])

    optimizer = keras.optimizers.AdamW(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# ====================================
# 5) Load or Create Model
# ====================================
if os.path.exists(latest_model_path):
    model = keras.models.load_model(latest_model_path)
    print("✅ Loaded latest model")
else:
    model = create_model()
    print("🚀 Created new model with pretrained EfficientNetB3 weights")

# ====================================
# 6) Training with Memory Fix
# ====================================
model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)
model.save(latest_model_path)
print("✅ Model saved!")


✅ Mixed precision enabled
📌 Detected 27 classes
Found 132025 files belonging to 27 classes.
Using 105620 files for training.
Found 132025 files belonging to 27 classes.
Using 26405 files for validation.
✅ Loaded EfficientNetB3 pretrained weights
🚀 Created new model with pretrained EfficientNetB3 weights
Epoch 1/3
  17/6602 ━━━━━━━━━━━━━━━━━━━━ 39:06:07 21s/step - accuracy: 0.0665 - loss: 5.6087

KeyboardInterrupt: 

In [4]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import mixed_precision
import datetime

# ====================================
# 1) Mixed Precision & Basic Setup
# ====================================
mixed_precision.set_global_policy('mixed_float16')
tf.config.optimizer.set_jit(True)  # Enable XLA compilation for faster training
print("Mixed precision enabled:", mixed_precision.global_policy())

# ====================================
# 2) CONFIGURATION & PATHS
# ====================================
image_dir = "/kaggle/input/trees-nineteen-dataset"
save_dir = "/kaggle/working"
os.makedirs(save_dir, exist_ok=True)

model_prefix = os.path.join(save_dir, "model_epoch_")
latest_model_path = os.path.join(save_dir, "latest_model.keras")
history_path = os.path.join(save_dir, "history.json")
class_indices_path = os.path.join(save_dir, "class_indices.json")
log_dir = os.path.join(save_dir, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

weights_file = "/kaggle/input/efficientnetb3/efficientnetb3_notop.h5"
if not os.path.exists(weights_file):
    raise FileNotFoundError("Weights file not found! Please upload EfficientNetB3 weights.")
else:
    print("Using local weights file:", weights_file)

# Reduce batch size to prevent RAM issues
batch_size = 16  # Reduced from 32 to avoid out-of-memory errors
input_shape = (224, 224, 3)
initial_epochs = 3
fine_tune_epochs = 5
base_learning_rate = 0.0005
fine_tune_learning_rate = 1e-6

# ====================================
# 3) DETECT CLASSES & BUILD CLASS INDICES
# ====================================
detected_classes = sorted(
    d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))
)

if os.path.exists(class_indices_path):
    with open(class_indices_path, "r") as f:
        class_indices = json.load(f)
else:
    class_indices = {name: i for i, name in enumerate(detected_classes)}
    with open(class_indices_path, "w") as f:
        json.dump(class_indices, f)
num_classes = len(class_indices)
print("Class indices:", class_indices)

# ====================================
# 4) SET UP DATA GENERATORS (OPTIMIZED)
# ====================================
from tensorflow.keras.applications.efficientnet import preprocess_input

AUTOTUNE = tf.data.AUTOTUNE  # Optimize data loading

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Normalizes to [-1, 1]
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode="nearest",
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=list(class_indices.keys()),
    workers=2,  # Use 2 workers for faster data loading
    use_multiprocessing=True  # Enable multiprocessing
)

val_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=list(class_indices.keys()),
    workers=2,
    use_multiprocessing=True
)

# Prefetch data for better performance
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32),
    )
).prefetch(buffer_size=AUTOTUNE)

val_dataset = tf.data.Dataset.from_generator(
    lambda: val_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32),
    )
).prefetch(buffer_size=AUTOTUNE)

# ====================================
# 5) DEFINE MODEL CREATION FUNCTION (OPTIMIZED)
# ====================================
def create_model(fine_tune=False):
    base_model = keras.applications.EfficientNetB3(
        include_top=False,
        weights=weights_file,
        input_shape=input_shape
    )

    if fine_tune:
        base_model.trainable = True
        fine_tune_at = len(base_model.layers) - 20  # Unfreeze last 20 layers (not 50)
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
        learning_rate = fine_tune_learning_rate
    else:
        base_model.trainable = False
        learning_rate = base_learning_rate

    model = keras.Sequential([
        keras.Input(shape=input_shape),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])

    optimizer = keras.optimizers.AdamW(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ====================================
# 6) LOAD OR CREATE MODEL
# ====================================
if os.path.exists(latest_model_path):
    model = keras.models.load_model(latest_model_path)
    print("✅ Loaded latest model from disk.")
else:
    model = create_model()
    print("🚀 Created new model from scratch.")

# ====================================
# 7) TRAINING FUNCTION WITH PER-EPOCH SAVING (OPTIMIZED)
# ====================================
def train_model(model, epochs, start_epoch=1):
    for epoch in range(start_epoch, start_epoch + epochs):
        print(f"\n🔄 Training epoch {epoch}/{start_epoch + epochs - 1}...")
        model.fit(train_dataset, validation_data=val_dataset, epochs=1)
        
        model_epoch_path = f"{model_prefix}{epoch}.keras"
        model.save(model_epoch_path)
        model.save(latest_model_path)
        print(f"✅ Model saved: {model_epoch_path}")
        
    print("✅ Training session complete!")

# ====================================
# 8) INITIAL TRAINING (SAVES EVERY EPOCH)
# ====================================
print("\n=== Initial Training Phase ===")
train_model(model, epochs=2)  # Run only 2 epochs per Kaggle session

# ====================================
# 9) HOW TO RESUME TRAINING NEXT TIME
# ====================================
# 1. Upload the latest saved model (e.g., model_epoch_2.keras) to Kaggle input.
# 2. Update 'latest_model_path' to point to the uploaded model.
# 3. Restart training from the next epoch.


Mixed precision enabled: <DTypePolicy "mixed_float16">
Using local weights file: /kaggle/input/efficientnetb3/efficientnetb3_notop.h5
Class indices: {'Azadirachta_indica': 0, 'Baccharis_salicifolia': 1, 'Betula_pendula': 2, 'Camphora_officinarum': 3, 'Ceiba_pentandra': 4, 'Corylus_avellana': 5, 'Delonix_regia': 6, 'Eucalyptus_globulus': 7, 'Ficus_carica': 8, 'Ficus_microcarpa': 9, 'Juglans_regia': 10, 'Leucaena_leucocephala': 11, 'Ligustrum_lucidum': 12, 'Ligustrum_ovalifolium': 13, 'Malus_domestica': 14, 'Mangifera_indica': 15, 'Murraya_paniculata': 16, 'Olea_europaea': 17, 'Paulownia_tomentosa': 18, 'Pinus_halepensis': 19, 'Psidium_guajava': 20, 'Spathodea_campanulata': 21, 'Tecoma_stans': 22, 'Tilia_cordata': 23, 'Ulmus_parvifolia': 24, 'Washingtonia_robusta': 25, 'unknown': 26}


TypeError: ImageDataGenerator.flow_from_directory() got an unexpected keyword argument 'workers'

In [5]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import mixed_precision
import datetime

# ====================================
# 1) Mixed Precision & Basic Setup
# ====================================
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision enabled:", mixed_precision.global_policy())

# ====================================
# 2) CONFIGURATION & PATHS
# ====================================
image_dir = "/kaggle/input/trees-nineteen-dataset"
save_dir = "/kaggle/working"
os.makedirs(save_dir, exist_ok=True)

model_prefix = os.path.join(save_dir, "model_epoch_")
latest_model_path = os.path.join(save_dir, "latest_model.keras")
history_path = os.path.join(save_dir, "history.json")
class_indices_path = os.path.join(save_dir, "class_indices.json")
log_dir = os.path.join(save_dir, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

weights_file = "/kaggle/input/efficientnetb3/efficientnetb3_notop.h5"
if not os.path.exists(weights_file):
    raise FileNotFoundError("Weights file not found! Please upload EfficientNetB3 weights.")
else:
    print("Using local weights file:", weights_file)

batch_size = 32
input_shape = (224, 224, 3)
initial_epochs = 3
fine_tune_epochs = 5
base_learning_rate = 0.0005
fine_tune_learning_rate = 1e-6

# ====================================
# 3) DETECT CLASSES & BUILD CLASS INDICES
# ====================================
detected_classes = sorted(
    d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))
)

if os.path.exists(class_indices_path):
    with open(class_indices_path, "r") as f:
        class_indices = json.load(f)
else:
    class_indices = {name: i for i, name in enumerate(detected_classes)}
    with open(class_indices_path, "w") as f:
        json.dump(class_indices, f)
num_classes = len(class_indices)
print("Class indices:", class_indices)

# ====================================
# 4) SET UP DATA GENERATORS
# ====================================
from tensorflow.keras.applications.efficientnet import preprocess_input

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Normalizes to [-1, 1]
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode="nearest",
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=list(class_indices.keys())  # Keep class order consistent
)

val_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=list(class_indices.keys())
)

# ====================================
# 5) DEFINE MODEL CREATION FUNCTION
# ====================================
def create_model(fine_tune=False):
    base_model = keras.applications.EfficientNetB3(
        include_top=False,
        weights=weights_file,
        input_shape=input_shape
    )

    if fine_tune:
        base_model.trainable = True
        fine_tune_at = len(base_model.layers) - 50  # Unfreeze last 50 layers
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
        learning_rate = fine_tune_learning_rate
    else:
        base_model.trainable = False
        learning_rate = base_learning_rate

    model = keras.Sequential([
        keras.Input(shape=input_shape),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])

    optimizer = keras.optimizers.AdamW(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ====================================
# 6) LOAD OR CREATE MODEL
# ====================================
if os.path.exists(latest_model_path):
    model = keras.models.load_model(latest_model_path)
    print("✅ Loaded latest model from disk.")
else:
    model = create_model()
    print("🚀 Created new model from scratch.")

# ====================================
# 7) TRAINING FUNCTION WITH PER-EPOCH SAVING
# ====================================
def train_model(model, epochs, start_epoch=1):
    for epoch in range(start_epoch, start_epoch + epochs):
        print(f"\n🔄 Training epoch {epoch}/{start_epoch + epochs - 1}...")
        model.fit(
            train_generator,
            validation_data=val_generator,
            epochs=1,
            workers=2,  # Multi-threaded data loading
            use_multiprocessing=True  # Reduce RAM usage
        )
        
        model_epoch_path = f"{model_prefix}{epoch}.keras"
        model.save(model_epoch_path)
        model.save(latest_model_path)
        print(f"✅ Model saved: {model_epoch_path}")
        
    print("✅ Training session complete!")

# ====================================
# 8) INITIAL TRAINING (SAVES EVERY EPOCH)
# ====================================
print("\n=== Initial Training Phase ===")
train_model(model, epochs=2)  # Run only 2 epochs per Kaggle session

# ====================================
# 9) HOW TO RESUME TRAINING NEXT TIME
# ====================================
# 1. Upload the latest saved model (e.g., model_epoch_2.keras) to Kaggle input.
# 2. Update 'latest_model_path' to point to the uploaded model.
# 3. Restart training from the next epoch.


Mixed precision enabled: <DTypePolicy "mixed_float16">
Using local weights file: /kaggle/input/efficientnetb3/efficientnetb3_notop.h5
Class indices: {'Azadirachta_indica': 0, 'Baccharis_salicifolia': 1, 'Betula_pendula': 2, 'Camphora_officinarum': 3, 'Ceiba_pentandra': 4, 'Corylus_avellana': 5, 'Delonix_regia': 6, 'Eucalyptus_globulus': 7, 'Ficus_carica': 8, 'Ficus_microcarpa': 9, 'Juglans_regia': 10, 'Leucaena_leucocephala': 11, 'Ligustrum_lucidum': 12, 'Ligustrum_ovalifolium': 13, 'Malus_domestica': 14, 'Mangifera_indica': 15, 'Murraya_paniculata': 16, 'Olea_europaea': 17, 'Paulownia_tomentosa': 18, 'Pinus_halepensis': 19, 'Psidium_guajava': 20, 'Spathodea_campanulata': 21, 'Tecoma_stans': 22, 'Tilia_cordata': 23, 'Ulmus_parvifolia': 24, 'Washingtonia_robusta': 25, 'unknown': 26}
Found 105631 images belonging to 27 classes.
Found 26394 images belonging to 27 classes.
🚀 Created new model from scratch.

=== Initial Training Phase ===

🔄 Training epoch 1/2...


TypeError: TensorFlowTrainer.fit() got an unexpected keyword argument 'workers'

In [6]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import mixed_precision
import datetime

# ====================================
# 1) Mixed Precision & Basic Setup
# ====================================
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision enabled:", mixed_precision.global_policy())

# ====================================
# 2) CONFIGURATION & PATHS
# ====================================
image_dir = "/kaggle/input/trees-nineteen-dataset"
save_dir = "/kaggle/working"
os.makedirs(save_dir, exist_ok=True)

model_prefix = os.path.join(save_dir, "model_epoch_")
latest_model_path = os.path.join(save_dir, "latest_model.keras")
history_path = os.path.join(save_dir, "history.json")
class_indices_path = os.path.join(save_dir, "class_indices.json")
log_dir = os.path.join(save_dir, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

weights_file = "/kaggle/input/efficientnetb3/efficientnetb3_notop.h5"
if not os.path.exists(weights_file):
    raise FileNotFoundError("Weights file not found! Please upload EfficientNetB3 weights.")
else:
    print("Using local weights file:", weights_file)

batch_size = 32
input_shape = (224, 224, 3)
initial_epochs = 3
fine_tune_epochs = 5
base_learning_rate = 0.0005
fine_tune_learning_rate = 1e-6

# ====================================
# 3) DETECT CLASSES & BUILD CLASS INDICES
# ====================================
detected_classes = sorted(
    d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))
)

if os.path.exists(class_indices_path):
    with open(class_indices_path, "r") as f:
        class_indices = json.load(f)
else:
    class_indices = {name: i for i, name in enumerate(detected_classes)}
    with open(class_indices_path, "w") as f:
        json.dump(class_indices, f)
num_classes = len(class_indices)
print("Class indices:", class_indices)

# ====================================
# 4) SET UP DATA GENERATORS
# ====================================
from tensorflow.keras.applications.efficientnet import preprocess_input

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Normalizes to [-1, 1]
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode="nearest",
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=list(class_indices.keys())  # Keep class order consistent
)

val_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=list(class_indices.keys())
)

steps_per_epoch = len(train_generator)  # Ensures controlled training on large datasets
validation_steps = len(val_generator)

# ====================================
# 5) DEFINE MODEL CREATION FUNCTION
# ====================================
def create_model(fine_tune=False):
    base_model = keras.applications.EfficientNetB3(
        include_top=False,
        weights=weights_file,
        input_shape=input_shape
    )

    if fine_tune:
        base_model.trainable = True
        fine_tune_at = len(base_model.layers) - 50  # Unfreeze last 50 layers
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
        learning_rate = fine_tune_learning_rate
    else:
        base_model.trainable = False
        learning_rate = base_learning_rate

    model = keras.Sequential([
        keras.Input(shape=input_shape),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])

    optimizer = keras.optimizers.AdamW(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ====================================
# 6) LOAD OR CREATE MODEL
# ====================================
if os.path.exists(latest_model_path):
    model = keras.models.load_model(latest_model_path)
    print("✅ Loaded latest model from disk.")
else:
    model = create_model()
    print("🚀 Created new model from scratch.")

# ====================================
# 7) TRAINING FUNCTION WITH PER-EPOCH SAVING
# ====================================
def train_model(model, epochs, start_epoch=1):
    for epoch in range(start_epoch, start_epoch + epochs):
        print(f"\n🔄 Training epoch {epoch}/{start_epoch + epochs - 1}...")
        model.fit(
            train_generator,
            validation_data=val_generator,
            epochs=1,
            steps_per_epoch=steps_per_epoch,
            validation_steps=validation_steps
        )
        
        model_epoch_path = f"{model_prefix}{epoch}.keras"
        model.save(model_epoch_path)
        model.save(latest_model_path)
        print(f"✅ Model saved: {model_epoch_path}")
        
    print("✅ Training session complete!")

# ====================================
# 8) INITIAL TRAINING (SAVES EVERY EPOCH)
# ====================================
print("\n=== Initial Training Phase ===")
train_model(model, epochs=2)  # Run only 2 epochs per Kaggle session

# ====================================
# 9) HOW TO RESUME TRAINING NEXT TIME
# ====================================
# 1. Upload the latest saved model (e.g., model_epoch_2.keras) to Kaggle input.
# 2. Update 'latest_model_path' to point to the uploaded model.
# 3. Restart training from the next epoch.


Mixed precision enabled: <DTypePolicy "mixed_float16">
Using local weights file: /kaggle/input/efficientnetb3/efficientnetb3_notop.h5
Class indices: {'Azadirachta_indica': 0, 'Baccharis_salicifolia': 1, 'Betula_pendula': 2, 'Camphora_officinarum': 3, 'Ceiba_pentandra': 4, 'Corylus_avellana': 5, 'Delonix_regia': 6, 'Eucalyptus_globulus': 7, 'Ficus_carica': 8, 'Ficus_microcarpa': 9, 'Juglans_regia': 10, 'Leucaena_leucocephala': 11, 'Ligustrum_lucidum': 12, 'Ligustrum_ovalifolium': 13, 'Malus_domestica': 14, 'Mangifera_indica': 15, 'Murraya_paniculata': 16, 'Olea_europaea': 17, 'Paulownia_tomentosa': 18, 'Pinus_halepensis': 19, 'Psidium_guajava': 20, 'Spathodea_campanulata': 21, 'Tecoma_stans': 22, 'Tilia_cordata': 23, 'Ulmus_parvifolia': 24, 'Washingtonia_robusta': 25, 'unknown': 26}
Found 105631 images belonging to 27 classes.
Found 26394 images belonging to 27 classes.
🚀 Created new model from scratch.

=== Initial Training Phase ===

🔄 Training epoch 1/2...


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


   8/3301 ━━━━━━━━━━━━━━━━━━━━ 9:50:06 11s/step - accuracy: 0.0709 - loss: 4.2025

KeyboardInterrupt: 

In [8]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.mixed_precision import set_global_policy

# Enable mixed precision for GPU acceleration
try:
    set_global_policy('mixed_float16')  # Remove this if using CPU
except:
    print("Mixed precision not available, using default float32.")

# Paths & parameters
image_dir = "/kaggle/input/trees-nineteen-dataset"
batch_size = 16  # Reduced to optimize RAM usage
input_shape = (300, 300, 3)  # Ensure resolution remains unchanged
num_classes = 27

# Load EfficientNetB3 without top layers
base_model = EfficientNetB3(weights='/kaggle/input/efficientnetb3/efficientnetb3_notop.h5', 
                            include_top=False, input_shape=input_shape)

# Freeze base model layers
base_model.trainable = False

# Add custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.4)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax', dtype='float32')(x)  # Ensure correct dtype

model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Data augmentation & generators
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=False,  # Disable shuffle to reduce RAM usage
)

val_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
)

# Training function to continue training across sessions
def train_model(model, epochs=2, start_epoch=1):
    for epoch in range(start_epoch, start_epoch + epochs):
        print(f"\n🔄 Training epoch {epoch}/{start_epoch + epochs - 1}...")
        model.fit(
            train_generator,
            validation_data=val_generator,
            epochs=1,  # Train 1 epoch per run to prevent crashes
            verbose=1
        )
        model.save(f"/kaggle/working/tree_classifier_epoch_{epoch}.h5")  # Save after each epoch

# Start training
print("\n=== Initial Training Phase ===")
train_model(model, epochs=2)  # Adjust this to fit within Kaggle limits


Found 105631 images belonging to 27 classes.
Found 26394 images belonging to 27 classes.

=== Initial Training Phase ===

🔄 Training epoch 1/2...
   3/6602 ━━━━━━━━━━━━━━━━━━━━ 18:03:50 10s/step - accuracy: 0.2639 - loss: 2.8660

KeyboardInterrupt: 

In [9]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import mixed_precision
import datetime

# ====================================
# 1) Mixed Precision & Basic Setup
# ====================================
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision enabled:", mixed_precision.global_policy())

# ====================================
# 2) CONFIGURATION & PATHS
# ====================================
image_dir = "/kaggle/input/trees-nineteen-dataset"
save_dir = "/kaggle/working"
os.makedirs(save_dir, exist_ok=True)

model_prefix = os.path.join(save_dir, "model_epoch_")
latest_model_path = os.path.join(save_dir, "latest_model.keras")
history_path = os.path.join(save_dir, "history.json")
class_indices_path = os.path.join(save_dir, "class_indices.json")
log_dir = os.path.join(save_dir, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

weights_file = "/kaggle/input/efficientnetb3/efficientnetb3_notop.h5"
if not os.path.exists(weights_file):
    raise FileNotFoundError("Weights file not found! Please upload EfficientNetB3 weights.")
else:
    print("Using local weights file:", weights_file)

batch_size = 16
input_shape = (224, 224, 3)
initial_epochs = 3
fine_tune_epochs = 5
base_learning_rate = 0.0005
fine_tune_learning_rate = 1e-6

# ====================================
# 3) DETECT CLASSES & BUILD CLASS INDICES
# ====================================
detected_classes = sorted(
    d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))
)

if os.path.exists(class_indices_path):
    with open(class_indices_path, "r") as f:
        class_indices = json.load(f)
else:
    class_indices = {name: i for i, name in enumerate(detected_classes)}
    with open(class_indices_path, "w") as f:
        json.dump(class_indices, f)
num_classes = len(class_indices)
print("Class indices:", class_indices)

# ====================================
# 4) SET UP DATA GENERATORS
# ====================================
from tensorflow.keras.applications.efficientnet import preprocess_input

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Normalizes to [-1, 1]
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode="nearest",
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=list(class_indices.keys())
)

val_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=list(class_indices.keys())
)

# ====================================
# 5) DEFINE MODEL CREATION FUNCTION
# ====================================
def create_model(fine_tune=False):
    base_model = keras.applications.EfficientNetB3(
        include_top=False,
        weights=weights_file,
        input_shape=input_shape
    )

    if fine_tune:
        base_model.trainable = True
        fine_tune_at = len(base_model.layers) - 50  # Unfreeze last 50 layers
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
        learning_rate = fine_tune_learning_rate
    else:
        base_model.trainable = False
        learning_rate = base_learning_rate

    model = keras.Sequential([
        keras.Input(shape=input_shape),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])

    optimizer = keras.optimizers.AdamW(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ====================================
# 6) LOAD OR CREATE MODEL
# ====================================
if os.path.exists(latest_model_path):
    model = keras.models.load_model(latest_model_path)
    print("✅ Loaded latest model from disk.")
else:
    model = create_model()
    print("🚀 Created new model from scratch.")

# ====================================
# 7) TRAINING FUNCTION WITH PER-EPOCH SAVING
# ====================================
def train_model(model, epochs, start_epoch=1):
    for epoch in range(start_epoch, start_epoch + epochs):
        print(f"\n🔄 Training epoch {epoch}/{start_epoch + epochs - 1}...")
        model.fit(train_generator, validation_data=val_generator, epochs=1)
        
        model_epoch_path = f"{model_prefix}{epoch}.keras"
        model.save(model_epoch_path)
        model.save(latest_model_path)
        print(f"✅ Model saved: {model_epoch_path}")
        
    print("✅ Training session complete!")

# , start_epoch=3
# ====================================
# 8) INITIAL TRAINING (SAVES EVERY EPOCH)
# ====================================
print("\n=== Initial Training Phase ===")
train_model(model, epochs=2)  # Run only 2 epochs per Kaggle session

# ====================================
# 9) HOW TO RESUME TRAINING NEXT TIME
# ====================================
# 1. Upload the latest saved model (e.g., model_epoch_2.keras) to Kaggle input.
# 2. Update 'latest_model_path' to point to the uploaded model.
# 3. Restart training from the next epoch.


Mixed precision enabled: <DTypePolicy "mixed_float16">
Using local weights file: /kaggle/input/efficientnetb3/efficientnetb3_notop.h5
Class indices: {'Azadirachta_indica': 0, 'Baccharis_salicifolia': 1, 'Betula_pendula': 2, 'Camphora_officinarum': 3, 'Ceiba_pentandra': 4, 'Corylus_avellana': 5, 'Delonix_regia': 6, 'Eucalyptus_globulus': 7, 'Ficus_carica': 8, 'Ficus_microcarpa': 9, 'Juglans_regia': 10, 'Leucaena_leucocephala': 11, 'Ligustrum_lucidum': 12, 'Ligustrum_ovalifolium': 13, 'Malus_domestica': 14, 'Mangifera_indica': 15, 'Murraya_paniculata': 16, 'Olea_europaea': 17, 'Paulownia_tomentosa': 18, 'Pinus_halepensis': 19, 'Psidium_guajava': 20, 'Spathodea_campanulata': 21, 'Tecoma_stans': 22, 'Tilia_cordata': 23, 'Ulmus_parvifolia': 24, 'Washingtonia_robusta': 25, 'unknown': 26}
Found 105631 images belonging to 27 classes.
Found 26394 images belonging to 27 classes.
🚀 Created new model from scratch.

=== Initial Training Phase ===

🔄 Training epoch 1/2...
  35/6602 ━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [11]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import mixed_precision
import datetime
import matplotlib.pyplot as plt

# ====================================
# 1) Mixed Precision & Basic Setup
# ====================================
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision enabled:", mixed_precision.global_policy())

# ====================================
# 2) CONFIGURATION & PATHS (Kaggle)
# ====================================
# Update these paths according to your Kaggle environment:
image_dir = "/kaggle/input/trees-nineteen-dataset"  # Dataset folder; each subfolder is a class.
save_dir = "/kaggle/working"                        # Model, history, and logs will be saved here.
os.makedirs(save_dir, exist_ok=True)

# We will save individual epoch models with a prefix and also keep a "latest" copy.
model_prefix = os.path.join(save_dir, "model_epoch_")
latest_model_path = os.path.join(save_dir, "latest_model.keras")
history_path = os.path.join(save_dir, "history.json")
class_indices_path = os.path.join(save_dir, "class_indices.json")
log_dir = os.path.join(save_dir, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

# IMPORTANT: Set the path to your local downloaded weights file.
# Upload your EfficientNetB3 weights file (e.g. "efficientnetb3_notop.h5") to the Kaggle working directory.
weights_file = "//kaggle/input/efficientnetb3/efficientnetb3_notop.h5"
if not os.path.exists(weights_file):
    raise FileNotFoundError("Weights file not found! Please upload your EfficientNetB3 weights file to: " + weights_file)
else:
    print("Using local weights file:", weights_file)

# Basic training parameters
batch_size = 32         # You can try reducing this (e.g., to 16) if you need faster epochs.
input_shape = (224, 224, 3)
base_learning_rate = 0.0005
initial_epochs = 3       # Number of epochs per training session in the initial phase
fine_tune_epochs = 5     # Number of epochs per fine-tuning session

# ====================================
# 3) DETECT CLASSES & BUILD CLASS INDICES
# ====================================
# Automatically detect classes from subfolders
detected_classes = sorted(
    d for d in os.listdir(image_dir)
    if os.path.isdir(os.path.join(image_dir, d))
)

# If a class indices JSON exists, update it; otherwise, create a new mapping.
if os.path.exists(class_indices_path):
    with open(class_indices_path, "r") as f:
        saved_class_indices = json.load(f)
    # Add any new classes
    for new_class in detected_classes:
        if new_class not in saved_class_indices:
            saved_class_indices[new_class] = len(saved_class_indices)
    class_indices = saved_class_indices
else:
    class_indices = {name: i for i, name in enumerate(detected_classes)}
    with open(class_indices_path, "w") as f:
        json.dump(class_indices, f)
num_classes = len(class_indices)
print("Updated class indices:", class_indices)

# ====================================
# 4) SET UP DATA GENERATORS
# ====================================
# Use EfficientNet's preprocess_input to normalize inputs to [-1, 1]
from tensorflow.keras.applications.efficientnet import preprocess_input

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode="nearest",
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=list(class_indices.keys())  # Fixes the class order
)

val_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=list(class_indices.keys())
)

# ====================================
# 5) DEFINE MODEL CREATION FUNCTION
# ====================================
def create_model(fine_tune=False):
    # Build the base using EfficientNetB3 with local weights
    base_model = keras.applications.EfficientNetB3(
        include_top=False,
        weights=weights_file,  # Use local weights
        input_shape=input_shape
    )
    if fine_tune:
        base_model.trainable = True
        # Unfreeze the last 30 layers; keep earlier layers frozen
        fine_tune_at = len(base_model.layers) - 30
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
        learning_rate = 1e-5  # Lower learning rate for fine-tuning
    else:
        base_model.trainable = False
        learning_rate = base_learning_rate

    model = keras.Sequential([
        keras.Input(shape=input_shape),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        # Explicitly output float32 for compatibility with mixed precision
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])
    optimizer = keras.optimizers.AdamW(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ====================================
# 6) LOAD OR CREATE MODEL
# ====================================
if os.path.exists(latest_model_path):
    model = keras.models.load_model(latest_model_path)
    print("✅ Loaded latest model from disk.")
else:
    model = create_model()
    print("🚀 Created new model from scratch.")

# ====================================
# 7) DEFINE CALLBACKS
# ====================================
checkpoint_callback = ModelCheckpoint(
    model_path,
    save_weights_only=False,
    save_best_only=False,  # Save every epoch
    verbose=1
)
early_stop = EarlyStopping(
    patience=5,
    restore_best_weights=True,
    monitor='val_loss'
)
reduce_lr = ReduceLROnPlateau(
    factor=0.5,
    patience=2,
    min_lr=1e-6,
    verbose=1
)
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks = [checkpoint_callback, early_stop, reduce_lr, tensorboard_callback]

# ====================================
# 8) TRAINING FUNCTION (WITH PER-EPOCH SAVING)
# ====================================
def train_model(model, epochs=3, start_epoch=1):
    for epoch in range(start_epoch, start_epoch + epochs):
        print(f"\n🔄 Training epoch {epoch}/{start_epoch + epochs - 1}...")
        # Train for one epoch at a time
        model.fit(
            train_generator,
            validation_data=val_generator,
            epochs=epoch + 1,
            initial_epoch=epoch
        )
        # Save model after each epoch
        model_epoch_path = f"{model_prefix}{epoch}.keras"
        model.save(model_epoch_path)
        model.save(latest_model_path)
        print(f"✅ Model saved: {model_epoch_path}")
    print("✅ Training session complete!")

# ====================================
# 9) MAIN TRAINING LOOP (INITIAL PHASE)
# ====================================
print("\n=== Initial Training Phase ===")
while True:
    ans = input("Run another initial training session? (yes/no): ")
    if ans.strip().lower() == "yes":
        train_model(model, epochs=3)  # Run 3 epochs per session
    else:
        break

# ====================================
# 10) FINE-TUNING PHASE (Optional)
# ====================================
ans = input("Proceed to fine-tuning phase? (yes/no): ")
if ans.strip().lower() == "yes":
    ft_model = create_model(fine_tune=True)
    ft_model.load_weights(latest_model_path)
    print("🔄 Starting fine-tuning with last 30 layers unfrozen.")
    train_model(ft_model, epochs=5)  # Fine-tune for 5 epochs per session
    print("✅ Fine-tuning complete!")


Mixed precision enabled: <DTypePolicy "mixed_float16">
Using local weights file: //kaggle/input/efficientnetb3/efficientnetb3_notop.h5
Updated class indices: {'Azadirachta_indica': 0, 'Baccharis_salicifolia': 1, 'Betula_pendula': 2, 'Camphora_officinarum': 3, 'Ceiba_pentandra': 4, 'Corylus_avellana': 5, 'Delonix_regia': 6, 'Eucalyptus_globulus': 7, 'Ficus_carica': 8, 'Ficus_microcarpa': 9, 'Juglans_regia': 10, 'Leucaena_leucocephala': 11, 'Ligustrum_lucidum': 12, 'Ligustrum_ovalifolium': 13, 'Malus_domestica': 14, 'Mangifera_indica': 15, 'Murraya_paniculata': 16, 'Olea_europaea': 17, 'Paulownia_tomentosa': 18, 'Pinus_halepensis': 19, 'Psidium_guajava': 20, 'Spathodea_campanulata': 21, 'Tecoma_stans': 22, 'Tilia_cordata': 23, 'Ulmus_parvifolia': 24, 'Washingtonia_robusta': 25, 'unknown': 26}
Found 105631 images belonging to 27 classes.
Found 26394 images belonging to 27 classes.
🚀 Created new model from scratch.


NameError: name 'model_path' is not defined

In [12]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import mixed_precision
import datetime

# Enable Mixed Precision & XLA for Speed Boost 🚀
mixed_precision.set_global_policy('mixed_float16')
tf.config.optimizer.set_jit(True)  # XLA Compilation

print("Mixed precision enabled:", mixed_precision.global_policy())

# ====================================
# 1) CONFIGURATION & PATHS
# ====================================
image_dir = "/kaggle/input/trees-nineteen-dataset"
save_dir = "/kaggle/working"
os.makedirs(save_dir, exist_ok=True)

model_prefix = os.path.join(save_dir, "model_epoch_")
latest_model_path = os.path.join(save_dir, "latest_model.keras")
history_path = os.path.join(save_dir, "history.json")
class_indices_path = os.path.join(save_dir, "class_indices.json")
log_dir = os.path.join(save_dir, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

weights_file = "/kaggle/input/efficientnetb3/efficientnetb3_notop.h5"
if not os.path.exists(weights_file):
    raise FileNotFoundError("Weights file not found! Please upload EfficientNetB3 weights.")
else:
    print("Using local weights file:", weights_file)

# Hyperparameters (Optimized for Speed ⚡)
batch_size = 32  # Increased from 16 to 32
input_shape = (224, 224, 3)
initial_epochs = 2  # Reduce epochs per session
fine_tune_epochs = 3  # Fine-tune with fewer epochs
base_learning_rate = 0.0005
fine_tune_learning_rate = 1e-5

# ====================================
# 2) DETECT CLASSES & BUILD CLASS INDICES
# ====================================
detected_classes = sorted(
    d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))
)

if os.path.exists(class_indices_path):
    with open(class_indices_path, "r") as f:
        class_indices = json.load(f)
else:
    class_indices = {name: i for i, name in enumerate(detected_classes)}
    with open(class_indices_path, "w") as f:
        json.dump(class_indices, f)

num_classes = len(class_indices)
print("Class indices:", class_indices)

# ====================================
# 3) DATA GENERATORS (Optimized)
# ====================================
from tensorflow.keras.applications.efficientnet import preprocess_input

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # EfficientNet built-in preprocessing
    rotation_range=20,  # Reduced for speed
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=list(class_indices.keys())
)

val_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=list(class_indices.keys())
)

# ====================================
# 4) DEFINE MODEL CREATION FUNCTION
# ====================================
def create_model(fine_tune=False):
    base_model = keras.applications.EfficientNetB3(
        include_top=False,
        weights=weights_file,
        input_shape=input_shape
    )

    if fine_tune:
        base_model.trainable = True
        fine_tune_at = len(base_model.layers) - 50  # Unfreeze last 50 layers
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
        learning_rate = fine_tune_learning_rate
    else:
        base_model.trainable = False
        learning_rate = base_learning_rate

    model = keras.Sequential([
        keras.Input(shape=input_shape),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.3),  # Reduced dropout for speed
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])

    optimizer = keras.optimizers.AdamW(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ====================================
# 5) LOAD OR CREATE MODEL
# ====================================
if os.path.exists(latest_model_path):
    model = keras.models.load_model(latest_model_path)
    print("✅ Loaded latest model from disk.")
else:
    model = create_model()
    print("🚀 Created new model from scratch.")

# ====================================
# 6) CALLBACKS FOR FASTER TRAINING
# ====================================
checkpoint_callback = ModelCheckpoint(
    latest_model_path,  # Save latest model
    save_weights_only=False,
    save_best_only=False,  # Save every epoch
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=2,  # Stop early if no improvement
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=1,
    verbose=1
)

tensorboard_callback = TensorBoard(log_dir=log_dir)

# ====================================
# 7) TRAINING FUNCTION WITH PER-EPOCH SAVING
# ====================================
def train_model(model, epochs, start_epoch=1):
    for epoch in range(start_epoch, start_epoch + epochs):
        print(f"\n🔄 Training epoch {epoch}/{start_epoch + epochs - 1}...")
        model.fit(
            train_generator,
            validation_data=val_generator,
            epochs=1,
            callbacks=[checkpoint_callback, reduce_lr, tensorboard_callback]
        )
        
        model_epoch_path = f"{model_prefix}{epoch}.keras"
        model.save(model_epoch_path)
        model.save(latest_model_path)
        print(f"✅ Model saved: {model_epoch_path}")
        
    print("✅ Training session complete!")

# ====================================
# 8) INITIAL TRAINING (SAVES EVERY EPOCH)
# ====================================
print("\n=== Initial Training Phase ===")
train_model(model, epochs=2)  # Reduced epochs to save time


Mixed precision enabled: <DTypePolicy "mixed_float16">
Using local weights file: /kaggle/input/efficientnetb3/efficientnetb3_notop.h5
Class indices: {'Azadirachta_indica': 0, 'Baccharis_salicifolia': 1, 'Betula_pendula': 2, 'Camphora_officinarum': 3, 'Ceiba_pentandra': 4, 'Corylus_avellana': 5, 'Delonix_regia': 6, 'Eucalyptus_globulus': 7, 'Ficus_carica': 8, 'Ficus_microcarpa': 9, 'Juglans_regia': 10, 'Leucaena_leucocephala': 11, 'Ligustrum_lucidum': 12, 'Ligustrum_ovalifolium': 13, 'Malus_domestica': 14, 'Mangifera_indica': 15, 'Murraya_paniculata': 16, 'Olea_europaea': 17, 'Paulownia_tomentosa': 18, 'Pinus_halepensis': 19, 'Psidium_guajava': 20, 'Spathodea_campanulata': 21, 'Tecoma_stans': 22, 'Tilia_cordata': 23, 'Ulmus_parvifolia': 24, 'Washingtonia_robusta': 25, 'unknown': 26}
Found 105631 images belonging to 27 classes.
Found 26394 images belonging to 27 classes.
🚀 Created new model from scratch.

=== Initial Training Phase ===

🔄 Training epoch 1/2...
  10/3301 ━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import mixed_precision
import datetime

# ====================================
# 1) Mixed Precision & Basic Setup
# ====================================
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision enabled:", mixed_precision.global_policy())

# ====================================
# 2) CONFIGURATION & PATHS
# ====================================
image_dir = "/kaggle/input/trees-nineteen-dataset"
save_dir = "/kaggle/working"
os.makedirs(save_dir, exist_ok=True)

model_prefix = os.path.join(save_dir, "model_epoch_")
latest_model_path = os.path.join(save_dir, "latest_model.keras")
history_path = os.path.join(save_dir, "history.json")
class_indices_path = os.path.join(save_dir, "class_indices.json")
log_dir = os.path.join(save_dir, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

weights_file = "/kaggle/input/efficientnetb2/efficientnetb2_notop.h5"
if not os.path.exists(weights_file):
    raise FileNotFoundError("Weights file not found! Please upload EfficientNetB2 weights.")
else:
    print("Using local weights file:", weights_file)

batch_size = 32
input_shape = (224, 224, 3)
initial_epochs = 3
fine_tune_epochs = 5
base_learning_rate = 0.0005
fine_tune_learning_rate = 1e-6

# ====================================
# 3) DETECT CLASSES & BUILD CLASS INDICES
# ====================================
detected_classes = sorted(
    d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))
)

if os.path.exists(class_indices_path):
    with open(class_indices_path, "r") as f:
        class_indices = json.load(f)
else:
    class_indices = {name: i for i, name in enumerate(detected_classes)}
    with open(class_indices_path, "w") as f:
        json.dump(class_indices, f)
num_classes = len(class_indices)
print("Class indices:", class_indices)

# ====================================
# 4) SET UP DATA GENERATORS
# ====================================
from tensorflow.keras.applications.efficientnet import preprocess_input

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Normalizes to [-1, 1]
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode="nearest",
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=list(class_indices.keys())
)

val_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=list(class_indices.keys())
)

# ====================================
# 5) DEFINE MODEL CREATION FUNCTION
# ====================================
def create_model(fine_tune=False):
    base_model = keras.applications.EfficientNetB2(
        include_top=False,
        weights=weights_file,
        input_shape=input_shape
    )

    if fine_tune:
        base_model.trainable = True
        fine_tune_at = len(base_model.layers) - 50  # Unfreeze last 50 layers
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
        learning_rate = fine_tune_learning_rate
    else:
        base_model.trainable = False
        learning_rate = base_learning_rate

    model = keras.Sequential([
        keras.Input(shape=input_shape),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])

    optimizer = keras.optimizers.AdamW(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ====================================
# 6) LOAD OR CREATE MODEL
# ====================================
if os.path.exists(latest_model_path):
    model = keras.models.load_model(latest_model_path)
    print("✅ Loaded latest model from disk.")
else:
    model = create_model()
    print("🚀 Created new model from scratch.")

# ====================================
# 7) TRAINING FUNCTION WITH PER-EPOCH SAVING
# ====================================
def train_model(model, epochs, start_epoch=1):
    for epoch in range(start_epoch, start_epoch + epochs):
        print(f"\n🔄 Training epoch {epoch}/{start_epoch + epochs - 1}...")
        model.fit(train_generator, validation_data=val_generator, epochs=1)
        
        model_epoch_path = f"{model_prefix}{epoch}.keras"
        model.save(model_epoch_path)
        model.save(latest_model_path)
        print(f"✅ Model saved: {model_epoch_path}")
        
    print("✅ Training session complete!")

# ====================================
# 8) INITIAL TRAINING (SAVES EVERY EPOCH)
# ====================================
print("\n=== Initial Training Phase ===")
train_model(model, epochs=2)  # Run only 2 epochs per Kaggle session


Mixed precision enabled: <DTypePolicy "mixed_float16">
Using local weights file: /kaggle/input/efficientnetb2/efficientnetb2_notop.h5
Class indices: {'Azadirachta_indica': 0, 'Baccharis_salicifolia': 1, 'Betula_pendula': 2, 'Camphora_officinarum': 3, 'Ceiba_pentandra': 4, 'Corylus_avellana': 5, 'Delonix_regia': 6, 'Eucalyptus_globulus': 7, 'Ficus_carica': 8, 'Ficus_microcarpa': 9, 'Juglans_regia': 10, 'Leucaena_leucocephala': 11, 'Ligustrum_lucidum': 12, 'Ligustrum_ovalifolium': 13, 'Malus_domestica': 14, 'Mangifera_indica': 15, 'Murraya_paniculata': 16, 'Olea_europaea': 17, 'Paulownia_tomentosa': 18, 'Pinus_halepensis': 19, 'Psidium_guajava': 20, 'Spathodea_campanulata': 21, 'Tecoma_stans': 22, 'Tilia_cordata': 23, 'Ulmus_parvifolia': 24, 'Washingtonia_robusta': 25, 'unknown': 26}
Found 105631 images belonging to 27 classes.
Found 26394 images belonging to 27 classes.
🚀 Created new model from scratch.

=== Initial Training Phase ===

🔄 Training epoch 1/2...


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  46/3301 ━━━━━━━━━━━━━━━━━━━━ 7:22:00 8s/step - accuracy: 0.0842 - loss: 4.0707

In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import mixed_precision
import datetime

# ====================================
# 1) Mixed Precision & Basic Setup
# ====================================
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision enabled:", mixed_precision.global_policy())

# ====================================
# 2) CONFIGURATION & PATHS
# ====================================
image_dir = "/kaggle/input/trees-nineteen-dataset"  # Update this if dataset path is different
save_dir = "/kaggle/working"
os.makedirs(save_dir, exist_ok=True)

model_prefix = os.path.join(save_dir, "model_epoch_")
latest_model_path = "/kaggle/input/latest-nineteen/latest_model1.keras"
history_path = os.path.join(save_dir, "history.json")
class_indices_path = os.path.join(save_dir, "class_indices.json")
log_dir = os.path.join(save_dir, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

# ✅ Use locally uploaded EfficientNetB1 weights
weights_file = "/kaggle/input/efficientnetb1-notop/efficientnetb1_notop.h5"  # Update the path if needed
if not os.path.exists(weights_file):
    raise FileNotFoundError("Weights file not found! Please upload EfficientNetB1 weights.")
else:
    print("Using local weights file:", weights_file)

batch_size = 32  # Adjust based on memory availability
input_shape = (224, 224, 3)
initial_epochs = 3
fine_tune_epochs = 5
base_learning_rate = 0.0005
fine_tune_learning_rate = 1e-6

# ====================================
# 3) DETECT CLASSES & BUILD CLASS INDICES
# ====================================
detected_classes = sorted(
    d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))
)

if os.path.exists(class_indices_path):
    with open(class_indices_path, "r") as f:
        class_indices = json.load(f)
else:
    class_indices = {name: i for i, name in enumerate(detected_classes)}
    with open(class_indices_path, "w") as f:
        json.dump(class_indices, f)
num_classes = len(class_indices)
print("Class indices:", class_indices)

# ====================================
# 4) SET UP DATA GENERATORS
# ====================================
from tensorflow.keras.applications.efficientnet import preprocess_input

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Normalizes images to [-1, 1]
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode="nearest",
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=list(class_indices.keys())
)

val_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=list(class_indices.keys())
)

# ====================================
# 5) DEFINE MODEL CREATION FUNCTION
# ====================================
def create_model(fine_tune=False):
    base_model = keras.applications.EfficientNetB1(
        include_top=False,
        weights=weights_file,
        input_shape=input_shape
    )

    if fine_tune:
        base_model.trainable = True
        fine_tune_at = len(base_model.layers) - 50  # Unfreeze last 50 layers
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
        learning_rate = fine_tune_learning_rate
    else:
        base_model.trainable = False
        learning_rate = base_learning_rate

    model = keras.Sequential([
        keras.Input(shape=input_shape),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])

    optimizer = keras.optimizers.AdamW(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ====================================
# 6) LOAD OR CREATE MODEL
# ====================================
if os.path.exists(latest_model_path):
    model = keras.models.load_model(latest_model_path)
    print("✅ Loaded latest model from disk.")
else:
    model = create_model()
    print("🚀 Created new model from scratch.")

# ====================================
# 7) TRAINING FUNCTION WITH PER-EPOCH SAVING
# ====================================
def train_model(model, epochs, start_epoch=1):
    for epoch in range(start_epoch, start_epoch + epochs):
        print(f"\n🔄 Training epoch {epoch}/{start_epoch + epochs - 1}...")
        model.fit(train_generator, validation_data=val_generator, epochs=1)
        
        model_epoch_path = f"{model_prefix}{epoch}.keras"
        model.save(model_epoch_path)
        # model.save(latest_model_path)
        latest_model_path = os.path.join(save_dir, "latest_model1.keras")
        model.save(latest_model_path)  # ✅ Save in /kaggle/working/

        print(f"✅ Model saved: {model_epoch_path}")
        
    print("✅ Training session complete!")

# ====================================
# 8) INITIAL TRAINING (SAVES EVERY EPOCH)
# ====================================
print("\n=== Initial Training Phase ===")
train_model(model, epochs=1, start_epoch=2)  # Run only 2 epochs per Kaggle session

# ====================================
# 9) HOW TO RESUME TRAINING NEXT TIME
# ====================================
# 1. Upload the latest saved model (e.g., model_epoch_2.keras) to Kaggle input.
# 2. Update 'latest_model_path' to point to the uploaded model.
# 3. Restart training from the next epoch.


Mixed precision enabled: <DTypePolicy "mixed_float16">
Using local weights file: /kaggle/input/efficientnetb1-notop/efficientnetb1_notop.h5
Class indices: {'Azadirachta_indica': 0, 'Baccharis_salicifolia': 1, 'Betula_pendula': 2, 'Camphora_officinarum': 3, 'Ceiba_pentandra': 4, 'Corylus_avellana': 5, 'Delonix_regia': 6, 'Eucalyptus_globulus': 7, 'Ficus_carica': 8, 'Ficus_microcarpa': 9, 'Juglans_regia': 10, 'Leucaena_leucocephala': 11, 'Ligustrum_lucidum': 12, 'Ligustrum_ovalifolium': 13, 'Malus_domestica': 14, 'Mangifera_indica': 15, 'Murraya_paniculata': 16, 'Olea_europaea': 17, 'Paulownia_tomentosa': 18, 'Pinus_halepensis': 19, 'Psidium_guajava': 20, 'Spathodea_campanulata': 21, 'Tecoma_stans': 22, 'Tilia_cordata': 23, 'Ulmus_parvifolia': 24, 'Washingtonia_robusta': 25, 'unknown': 26}
Found 105631 images belonging to 27 classes.
Found 26394 images belonging to 27 classes.
✅ Loaded latest model from disk.

=== Initial Training Phase ===

🔄 Training epoch 2/2...


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1452/3301 ━━━━━━━━━━━━━━━━━━━━ 3:42:04 7s/step - accuracy: 0.4218 - loss: 1.9599

In [1]:
import os
image_dir = "/kaggle/input/trees-nineteen-dataset"
print("Classes detected:", sorted(os.listdir(image_dir)))
!pip install tensorflow keras numpy


Classes detected: ['Azadirachta_indica', 'Baccharis_salicifolia', 'Betula_pendula', 'Camphora_officinarum', 'Ceiba_pentandra', 'Corylus_avellana', 'Delonix_regia', 'Eucalyptus_globulus', 'Ficus_carica', 'Ficus_microcarpa', 'Juglans_regia', 'Leucaena_leucocephala', 'Ligustrum_lucidum', 'Ligustrum_ovalifolium', 'Malus_domestica', 'Mangifera_indica', 'Murraya_paniculata', 'Olea_europaea', 'Paulownia_tomentosa', 'Pinus_halepensis', 'Psidium_guajava', 'Spathodea_campanulata', 'Tecoma_stans', 'Tilia_cordata', 'Ulmus_parvifolia', 'Washingtonia_robusta', 'unknown']


In [2]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import mixed_precision
import datetime

# ====================================
# 1) Mixed Precision & Basic Setup
# ====================================
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision enabled:", mixed_precision.global_policy())

# ====================================
# 2) CONFIGURATION & PATHS
# ====================================
image_dir = "/kaggle/input/trees-nineteen-dataset"  # Your dataset folder
save_dir = "/kaggle/working"  # Writable directory for saving checkpoints
os.makedirs(save_dir, exist_ok=True)

model_prefix = os.path.join(save_dir, "model_epoch_")
# For loading, use your uploaded model (in Kaggle input). Replace "latest-nineteen" with your actual folder.
uploaded_model_path = "/kaggle/input/latest-nineteen/latest_model1.keras"
# For saving, always use the working directory:
latest_model_path = os.path.join(save_dir, "latest_model1.keras")
history_path = os.path.join(save_dir, "history.json")
class_indices_path = os.path.join(save_dir, "class_indices.json")
log_dir = os.path.join(save_dir, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

# ✅ Use locally uploaded EfficientNetB1 weights
weights_file = "/kaggle/input/efficientnetb1-notop/efficientnetb1_notop.h5"  # Update path if needed
if not os.path.exists(weights_file):
    raise FileNotFoundError("Weights file not found! Please upload EfficientNetB1 weights.")
else:
    print("Using local weights file:", weights_file)

batch_size = 32  # Adjust based on memory availability
input_shape = (224, 224, 3)
initial_epochs = 3
fine_tune_epochs = 5
base_learning_rate = 0.0005
fine_tune_learning_rate = 1e-6

# ====================================
# 3) DETECT CLASSES & BUILD CLASS INDICES
# ====================================
detected_classes = sorted(
    d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))
)

if os.path.exists(class_indices_path):
    with open(class_indices_path, "r") as f:
        class_indices = json.load(f)
else:
    class_indices = {name: i for i, name in enumerate(detected_classes)}
    with open(class_indices_path, "w") as f:
        json.dump(class_indices, f)
num_classes = len(class_indices)
print("Class indices:", class_indices)

# ====================================
# 4) SET UP DATA GENERATORS
# ====================================
from tensorflow.keras.applications.efficientnet import preprocess_input

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Normalizes images to [-1, 1]
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode="nearest",
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=list(class_indices.keys())
)

val_generator = datagen.flow_from_directory(
    directory=image_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=list(class_indices.keys())
)

# ====================================
# 5) DEFINE MODEL CREATION FUNCTION
# ====================================
def create_model(fine_tune=False):
    base_model = keras.applications.EfficientNetB1(
        include_top=False,
        weights=weights_file,
        input_shape=input_shape
    )

    if fine_tune:
        base_model.trainable = True
        fine_tune_at = len(base_model.layers) - 50  # Unfreeze last 50 layers
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
        learning_rate = fine_tune_learning_rate
    else:
        base_model.trainable = False
        learning_rate = base_learning_rate

    model = keras.Sequential([
        keras.Input(shape=input_shape),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])

    optimizer = keras.optimizers.AdamW(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ====================================
# 6) LOAD OR CREATE MODEL
# ====================================
# Try to load the uploaded model first (from /kaggle/input)
if os.path.exists(uploaded_model_path):
    model = keras.models.load_model(uploaded_model_path)
    print("✅ Loaded latest model from uploaded file.")
else:
    model = create_model()
    print("🚀 Created new model from scratch.")

# ====================================
# 7) TRAINING FUNCTION WITH PER-EPOCH SAVING
# ====================================
def train_model_func(model, epochs, start_epoch=1):
    # Note: latest_model_path is defined above for saving in /kaggle/working/
    for epoch in range(start_epoch, start_epoch + epochs):
        print(f"\n🔄 Training epoch {epoch}/{start_epoch + epochs - 1}...")
        model.fit(train_generator, validation_data=val_generator, epochs=1)
        
        model_epoch_path = f"{model_prefix}{epoch}.keras"
        model.save(model_epoch_path)
        model.save(latest_model_path)  # Save latest model in /kaggle/working/
        print(f"✅ Model saved: {model_epoch_path}")
        
    print("✅ Training session complete!")

# ====================================
# 8) INITIAL TRAINING PHASE
# ====================================
print("\n=== Training Phase ===")
# For example, we resume training starting from epoch 2.
train_model_func(model, epochs=1, start_epoch=2)

# ====================================
# 9) HOW TO RESUME TRAINING NEXT TIME
# ====================================
# Before the session ends:
#   1. Download the file at /kaggle/working/latest_model1.keras.
# Next time:
#   2. Upload the downloaded model to your Kaggle dataset (e.g., latest-nineteen).
#   3. The above code will load the uploaded model from /kaggle/input/latest-nineteen/latest_model1.keras.


Mixed precision enabled: <DTypePolicy "mixed_float16">
Using local weights file: /kaggle/input/efficientnetb1-notop/efficientnetb1_notop.h5
Class indices: {'Azadirachta_indica': 0, 'Baccharis_salicifolia': 1, 'Betula_pendula': 2, 'Camphora_officinarum': 3, 'Ceiba_pentandra': 4, 'Corylus_avellana': 5, 'Delonix_regia': 6, 'Eucalyptus_globulus': 7, 'Ficus_carica': 8, 'Ficus_microcarpa': 9, 'Juglans_regia': 10, 'Leucaena_leucocephala': 11, 'Ligustrum_lucidum': 12, 'Ligustrum_ovalifolium': 13, 'Malus_domestica': 14, 'Mangifera_indica': 15, 'Murraya_paniculata': 16, 'Olea_europaea': 17, 'Paulownia_tomentosa': 18, 'Pinus_halepensis': 19, 'Psidium_guajava': 20, 'Spathodea_campanulata': 21, 'Tecoma_stans': 22, 'Tilia_cordata': 23, 'Ulmus_parvifolia': 24, 'Washingtonia_robusta': 25, 'unknown': 26}
Found 105631 images belonging to 27 classes.
Found 26394 images belonging to 27 classes.
✅ Loaded latest model from uploaded file.

=== Training Phase ===

🔄 Training epoch 2/2...


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3301/3301 ━━━━━━━━━━━━━━━━━━━━ 32934s 10s/step - accuracy: 0.4269 - loss: 1.9394 - val_accuracy: 0.5244 - val_loss: 1.5843
✅ Model saved: /kaggle/working/model_epoch_2.keras
✅ Training session complete!


In [3]:
import shutil

# Zip the model file (replace with your filename)
shutil.make_archive('model_checkpoint', 'zip', '/kaggle/working/')

# Now download
from IPython.display import FileLink
FileLink(r'model_checkpoint.zip')


RuntimeError: File size unexpectedly exceeded ZIP64 limit

In [5]:
import shutil

# Create a ZIP file with only the model files
shutil.make_archive('/kaggle/working/my_models', 'zip', '/kaggle/working', 
                     base_dir=None, verbose=True)

# Generate a download link
FileLink('/kaggle/working/my_models.zip')


/kaggle/working/my_models.zip

In [6]:
import shutil

shutil.copy('/kaggle/working/latest_model1.keras', '/kaggle/input/latest_model1.keras')
shutil.copy('/kaggle/working/model_epoch_2.keras', '/kaggle/input/model_epoch_2.keras')


OSError: [Errno 30] Read-only file system: '/kaggle/input/latest_model1.keras'

In [11]:
!ls -lh /kaggle/working/


total 79M
-rw-r--r-- 1 root root 643 Feb 28 04:32 class_indices.json
-rw-r--r-- 1 root root 40M Feb 28 13:46 latest_model1.keras
-rw-r--r-- 1 root root 40M Feb 28 13:46 model_epoch_2.keras


In [10]:
import os

os.remove('/kaggle/working/model_checkpoint.zip')  # Delete the large ZIP


In [12]:
import zipfile

# Define the zip file name
zip_filename = "/kaggle/working/my_models_fixed.zip"

# Create a zip file and add only the required models
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    zipf.write('/kaggle/working/latest_model1.keras', arcname='latest_model1.keras')
    zipf.write('/kaggle/working/model_epoch_2.keras', arcname='model_epoch_2.keras')

# Generate a download link
from IPython.display import FileLink
FileLink(zip_filename)


/kaggle/working/my_models_fixed.zip

In [14]:
!ls -lh /kaggle/working/


total 157M
-rw-r--r-- 1 root root 643 Feb 28 04:32 class_indices.json
-rw-r--r-- 1 root root 40M Feb 28 13:46 latest_model1.keras
-rw-r--r-- 1 root root 40M Feb 28 13:46 model_epoch_2.keras
-rw-r--r-- 1 root root 79M Feb 28 14:01 my_models_fixed.zip


In [15]:
from google.colab import drive
drive.mount('/content/drive')


NotImplementedError: Mounting drive is unsupported in this environment. Use PyDrive instead. See examples at https://colab.research.google.com/notebooks/io.ipynb#scrollTo=7taylj9wpsA2.

In [16]:
import zipfile

zip_filename = "/kaggle/working/my_models_fixed_nineteen.zip"

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    zipf.write('/kaggle/working/latest_model1.keras', arcname='latest_model1.keras')
    zipf.write('/kaggle/working/model_epoch_2.keras', arcname='model_epoch_2.keras')

print("✅ Zip file created successfully!")


✅ Zip file created successfully!


In [17]:
!cp /kaggle/working/my_models_fixed.zip /kaggle/working/


cp: '/kaggle/working/my_models_fixed.zip' and '/kaggle/working/my_models_fixed.zip' are the same file


In [18]:
!pip install kaggle


In [19]:
!mkdir -p /kaggle/working/kaggle-dataset


In [20]:
meta_data = '''{
  "title": "My Saved Models",
  "id": "your-username/my-saved-models",
  "licenses": [{"name": "CC0-1.0"}]
}'''

with open("/kaggle/working/kaggle-dataset/dataset-metadata.json", "w") as f:
    f.write(meta_data)


In [21]:
!mv /kaggle/working/*.keras /kaggle/working/kaggle-dataset/
!mv /kaggle/working/*.zip /kaggle/working/kaggle-dataset/


In [22]:
!kaggle datasets create -p /kaggle/working/kaggle-dataset/ --public


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [23]:
!mkdir -p /root/.kaggle
!mv /kaggle/working/kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


mv: cannot stat '/kaggle/working/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [25]:
!mkdir -p /root/.kaggle
!cp "/kaggle/input/kaggle/kaggle (1).json" /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json


In [26]:
!kaggle datasets list


2025-02-28 14:25:47,909 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x792c38b93670>: Failed to resolve 'www.kaggle.com' ([Errno -3] Temporary failure in name resolution)")': /api/v1/datasets/list?group=public&sortBy=hottest&size=None&filetype=all&license=all&tagids=&search=&user=&page=1&maxSize=None&minSize=None
2025-02-28 14:26:19,921 WARNING Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x792c38b936a0>: Failed to resolve 'www.kaggle.com' ([Errno -3] Temporary failure in name resolution)")': /api/v1/datasets/list?group=public&sortBy=hottest&size=None&filetype=all&license=all&tagids=&search=&user=&page=1&maxSize=None&minSize=None
^C


In [27]:
!kaggle datasets create -p /kaggle/working/kaggle-dataset/ --public


Starting upload for file latest_model1.keras
2025-02-28 14:27:02,586 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7a6afd754e50>: Failed to resolve 'www.kaggle.com' ([Errno -3] Temporary failure in name resolution)")': /api/v1/blobs/upload
^C


In [28]:
!kaggle datasets create -p /kaggle/working/kaggle-dataset/ --private


usage: kaggle [-h] [-v] [-W] {competitions,c,datasets,d,kernels,k,models,m,files,f,config} ...
kaggle: error: unrecognized arguments: --private


In [29]:
!zip -r /kaggle/working/my_models.zip /kaggle/working/kaggle-dataset


  adding: kaggle/working/kaggle-dataset/ (stored 0%)
  adding: kaggle/working/kaggle-dataset/dataset-metadata.json (deflated 16%)
  adding: kaggle/working/kaggle-dataset/latest_model1.keras (deflated 11%)
  adding: kaggle/working/kaggle-dataset/my_models_fixed_nineteen.zip (stored 0%)
  adding: kaggle/working/kaggle-dataset/model_epoch_2.keras (deflated 11%)
  adding: kaggle/working/kaggle-dataset/my_models_fixed.zip (stored 0%)


In [ ]:
!kaggle datasets create -p /kaggle/working/kaggle-dataset --public
